In [2]:
import numpy as np
import pandas as pd


%time
import os
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import sklearn

# Libraries and packages for text (pre-)processing 
import string
import re
import nltk


CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head()
#test.head()

,ID,TITLE,DATE,INFO,TEXT,Depression
0,test_subject1005,NaN,2015-03-18 23:49:34,reddit post,"""Kill Bill Siren"" was awesome. These seriously...",0
1,test_subject1005,NaN,2015-03-17 21:22:47,reddit post,crazy bastard,0
2,test_subject1005,NaN,2015-03-17 17:30:23,reddit post,What I hoped for this post: something other th...,0
3,test_subject1005,NaN,2015-03-16 21:55:42,reddit post,"I didn't find this scene funny, but the fact t...",0
4,test_subject1005,NaN,2015-03-16 21:36:35,reddit post,Bill from Kill Bill. Not saying the five point...,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531394 entries, 0 to 531393
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          531394 non-null  object
 1   TITLE       163955 non-null  object
 2   DATE        531394 non-null  object
 3   INFO        531394 non-null  object
 4   TEXT        389544 non-null  object
 5   Depression  531394 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 24.3+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544447 entries, 0 to 544446
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          544447 non-null  object
 1   TITLE       177602 non-null  object
 2   DATE        544447 non-null  object
 3   INFO        544447 non-null  object
 4   TEXT        397250 non-null  object
 5   Depression  544447 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 24.9+ MB


In [7]:
train['TEXT'] = train['TITLE'].fillna('-') + " " + train['TEXT'].fillna('-')
test['TEXT'] = test['TITLE'].fillna('-') + " " + test['TEXT'].fillna('-')

In [8]:
train.drop(['DATE','INFO','TITLE'],inplace= True,axis=1)
test.drop(['DATE','INFO','TITLE'],inplace= True,axis=1)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531394 entries, 0 to 531393
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          531394 non-null  object
 1   TEXT        531394 non-null  object
 2   Depression  531394 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.2+ MB


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544447 entries, 0 to 544446
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          544447 non-null  object
 1   TEXT        544447 non-null  object
 2   Depression  544447 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.5+ MB


In [11]:
train.head()

,ID,TEXT,Depression
0,test_subject1005,"- ""Kill Bill Siren"" was awesome. These serious...",0
1,test_subject1005,- crazy bastard,0
2,test_subject1005,- What I hoped for this post: something other ...,0
3,test_subject1005,"- I didn't find this scene funny, but the fact...",0
4,test_subject1005,- Bill from Kill Bill. Not saying the five poi...,0


#  text preprocess

In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop = stopwords.words('english')


## lower_case

In [14]:
train['TEXT'][4]

"- Bill from Kill Bill. Not saying the five point palm exploding heart technique sucks....but he's a dude we have waited through TWO movies to see die. And he falls. On the grass. Cool."

In [15]:
def lower_case(df):
    df['TEXT'] = df['TEXT'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    print(df['TEXT'].head())

In [16]:
lower_case(train)
lower_case(test)

0    - "kill bill siren" was awesome. these serious...
1                                      - crazy bastard
2    - what i hoped for this post: something other ...
3    - i didn't find this scene funny, but the fact...
4    - bill from kill bill. not saying the five poi...
Name: TEXT, dtype: object
0    til that saigas, a species of antelope with un...
1    what did bernie sanders say when he found a de...
2    - i didn't have any trouble with faux iosefka ...
3    - did you have trouble with yurie and faux ios...
4    - ;) what about the bloody crow of cainhurst? ...
Name: TEXT, dtype: object


In [17]:
train['TEXT'][3]

"- i didn't find this scene funny, but the fact that you did kills me."

## Expand the Contractions

In [18]:
import contractions
def expand_contractions(df):
    df['TEXT'] = df['TEXT'].apply(lambda x: contractions.fix(x))
    print(df['TEXT'].head())

In [19]:
train['TEXT'][3]

"- i didn't find this scene funny, but the fact that you did kills me."

In [20]:
expand_contractions(train)
expand_contractions(test)

0    - "kill bill siren" was awesome. these serious...
1                                      - crazy bastard
2    - what i hoped for this post: something other ...
3    - i did not find this scene funny, but the fac...
4    - bill from kill bill. not saying the five poi...
Name: TEXT, dtype: object
0    til that saigas, a species of antelope with un...
1    what did bernie sanders say when he found a de...
2    - i did not have any trouble with faux iosefka...
3    - did you have trouble with yurie and faux ios...
4    - ;) what about the bloody crow of cainhurst? ...
Name: TEXT, dtype: object


In [21]:
train['TEXT'][3]

'- i did not find this scene funny, but the fact that you did kills me.'

## noisy removal

## remove urls

In [22]:
train['TEXT'][278]

'- its the same format used in https://www.reddit.com/r/redditwritesseinfeld'

In [23]:
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)



def url_remove(df):
    df['TEXT'] = df['TEXT'].apply(lambda x: remove_URL(x))
    print(df['TEXT'].head())

In [24]:
url_remove(train)
url_remove(test)

0    - "kill bill siren" was awesome. these serious...
1                                      - crazy bastard
2    - what i hoped for this post: something other ...
3    - i did not find this scene funny, but the fac...
4    - bill from kill bill. not saying the five poi...
Name: TEXT, dtype: object
0    til that saigas, a species of antelope with un...
1    what did bernie sanders say when he found a de...
2    - i did not have any trouble with faux iosefka...
3    - did you have trouble with yurie and faux ios...
4    - ;) what about the bloody crow of cainhurst? ...
Name: TEXT, dtype: object


In [25]:
train['TEXT'][278]

'- its the same format used in '

## HTML tags remove

In [26]:
train['TEXT'][278]

'- its the same format used in '

In [27]:
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

def html_remove(df):
    df['TEXT'] = df['TEXT'].apply(lambda x: remove_html(x))
    print(df['TEXT'].head())

In [28]:
html_remove(train)
html_remove(test)

0    - "kill bill siren" was awesome. these serious...
1                                      - crazy bastard
2    - what i hoped for this post: something other ...
3    - i did not find this scene funny, but the fac...
4    - bill from kill bill. not saying the five poi...
Name: TEXT, dtype: object
0    til that saigas, a species of antelope with un...
1    what did bernie sanders say when he found a de...
2    - i did not have any trouble with faux iosefka...
3    - did you have trouble with yurie and faux ios...
4    - ;) what about the bloody crow of cainhurst? ...
Name: TEXT, dtype: object


## punctuation_removal

In [29]:
def punctuation_removal(df):
    df['TEXT'] = df['TEXT'].str.replace('[^\w\s]','')
    print(df['TEXT'].head())

In [30]:
train['TEXT'][0]

'- "kill bill siren" was awesome. these seriously make my week. you should do them for better call saul!!! that subreddit would be extremely grateful.'

In [31]:
punctuation_removal(train)
punctuation_removal(test)

0     kill bill siren was awesome these seriously m...
1                                        crazy bastard
2     what i hoped for this post something other th...
3     i did not find this scene funny but the fact ...
4     bill from kill bill not saying the five point...
Name: TEXT, dtype: object
0    til that saigas a species of antelope with unu...
1    what did bernie sanders say when he found a de...
2     i did not have any trouble with faux iosefka ...
3     did you have trouble with yurie and faux iose...
4      what about the bloody crow of cainhurst not ...
Name: TEXT, dtype: object


In [32]:
train['TEXT'][0]

' kill bill siren was awesome these seriously make my week you should do them for better call saul that subreddit would be extremely grateful'

# Replace typos,slang,acronyms or informal abbreviations

In [33]:
train['TEXT'][2154]

' mahouka sure tries hard to make the antidiscrimination course 2 students look like extremists fuck them not being happy with discrimination amirite there is a difference between institutionalized discrimination and personal bigotry they seemed to be fighting against the latter when the former is far more important the former could be changed with progressive policies while the latter cannot obviously bigotry is not ok but like mayumi said you cannot change what an individual feels by force'

In [34]:
def other_clean(text):
        # Typos, slang and other
        sample_typos_slang = {
                                "w/e": "whatever",
                                "usagov": "usa government",
                                "recentlu": "recently",
                                "ph0tos": "photos",
                                "amirite": "am i right",
                                "exp0sed": "exposed",
                                "<3": "love",
                                "luv": "love",
                                "amageddon": "armageddon",
                                "trfc": "traffic",
                                "16yr": "16 year"
                                }

        # Acronyms
        sample_acronyms =  { 
                            "mh370": "malaysia airlines flight 370",
                            "okwx": "oklahoma city weather",
                            "arwx": "arkansas weather",    
                            "gawx": "georgia weather",  
                            "scwx": "south carolina weather",  
                            "cawx": "california weather",
                            "tnwx": "tennessee weather",
                            "azwx": "arizona weather",  
                            "alwx": "alabama weather",
                            "usnwsgov": "united states national weather service",
                            "2mw": "tomorrow"
                            }

        
        # Some common abbreviations 
        sample_abbr = {
                        "$" : " dollar ",
                        "€" : " euro ",
                        "4ao" : "for adults only",
                        "a.m" : "before midday",
                        "a3" : "anytime anywhere anyplace",
                        "aamof" : "as a matter of fact",
                        "acct" : "account",
                        "adih" : "another day in hell",
                        "afaic" : "as far as i am concerned",
                        "afaict" : "as far as i can tell",
                        "afaik" : "as far as i know",
                        "afair" : "as far as i remember",
                        "afk" : "away from keyboard",
                        "app" : "application",
                        "approx" : "approximately",
                        "apps" : "applications",
                        "asap" : "as soon as possible",
                        "asl" : "age, sex, location",
                        "atk" : "at the keyboard",
                        "ave." : "avenue",
                        "aymm" : "are you my mother",
                        "ayor" : "at your own risk", 
                        "b&b" : "bed and breakfast",
                        "b+b" : "bed and breakfast",
                        "b.c" : "before christ",
                        "b2b" : "business to business",
                        "b2c" : "business to customer",
                        "b4" : "before",
                        "b4n" : "bye for now",
                        "b@u" : "back at you",
                        "bae" : "before anyone else",
                        "bak" : "back at keyboard",
                        "bbbg" : "bye bye be good",
                        "bbc" : "british broadcasting corporation",
                        "bbias" : "be back in a second",
                        "bbl" : "be back later",
                        "bbs" : "be back soon",
                        "be4" : "before",
                        "bfn" : "bye for now",
                        "blvd" : "boulevard",
                        "bout" : "about",
                        "brb" : "be right back",
                        "bros" : "brothers",
                        "brt" : "be right there",
                        "bsaaw" : "big smile and a wink",
                        "btw" : "by the way",
                        "bwl" : "bursting with laughter",
                        "c/o" : "care of",
                        "cet" : "central european time",
                        "cf" : "compare",
                        "cia" : "central intelligence agency",
                        "csl" : "can not stop laughing",
                        "cu" : "see you",
                        "cul8r" : "see you later",
                        "cv" : "curriculum vitae",
                        "cwot" : "complete waste of time",
                        "cya" : "see you",
                        "cyt" : "see you tomorrow",
                        "dae" : "does anyone else",
                        "dbmib" : "do not bother me i am busy",
                        "diy" : "do it yourself",
                        "dm" : "direct message",
                        "dwh" : "during work hours",
                        "e123" : "easy as one two three",
                        "eet" : "eastern european time",
                        "eg" : "example",
                        "embm" : "early morning business meeting",
                        "encl" : "enclosed",
                        "encl." : "enclosed",
                        "etc" : "and so on",
                        "faq" : "frequently asked questions",
                        "fawc" : "for anyone who cares",
                        "fb" : "facebook",
                        "fc" : "fingers crossed",
                        "fig" : "figure",
                        "fimh" : "forever in my heart", 
                        "ft." : "feet",
                        "ft" : "featuring",
                        "ftl" : "for the loss",
                        "ftw" : "for the win",
                        "fwiw" : "for what it is worth",
                        "fyi" : "for your information",
                        "g9" : "genius",
                        "gahoy" : "get a hold of yourself",
                        "gal" : "get a life",
                        "gcse" : "general certificate of secondary education",
                        "gfn" : "gone for now",
                        "gg" : "good game",
                        "gl" : "good luck",
                        "glhf" : "good luck have fun",
                        "gmt" : "greenwich mean time",
                        "gmta" : "great minds think alike",
                        "gn" : "good night",
                        "g.o.a.t" : "greatest of all time",
                        "goat" : "greatest of all time",
                        "goi" : "get over it",
                        "gps" : "global positioning system",
                        "gr8" : "great",
                        "gratz" : "congratulations",
                        "gyal" : "girl",
                        "h&c" : "hot and cold",
                        "hp" : "horsepower",
                        "hr" : "hour",
                        "hrh" : "his royal highness",
                        "ht" : "height",
                        "ibrb" : "i will be right back",
                        "ic" : "i see",
                        "icq" : "i seek you",
                        "icymi" : "in case you missed it",
                        "idc" : "i do not care",
                        "idgadf" : "i do not give a damn fuck",
                        "idgaf" : "i do not give a fuck",
                        "idk" : "i do not know",
                        "ie" : "that is",
                        "i.e" : "that is",
                        "ifyp" : "i feel your pain",
                        "IG" : "instagram",
                        "iirc" : "if i remember correctly",
                        "ilu" : "i love you",
                        "ily" : "i love you",
                        "imho" : "in my humble opinion",
                        "imo" : "in my opinion",
                        "imu" : "i miss you",
                        "iow" : "in other words",
                        "irl" : "in real life",
                        "j4f" : "just for fun",
                        "jic" : "just in case",
                        "jk" : "just kidding",
                        "jsyk" : "just so you know",
                        "l8r" : "later",
                        "lb" : "pound",
                        "lbs" : "pounds",
                        "ldr" : "long distance relationship",
                        "lmao" : "laugh my ass off",
                        "lmfao" : "laugh my fucking ass off",
                        "lol" : "laughing out loud",
                        "ltd" : "limited",
                        "ltns" : "long time no see",
                        "m8" : "mate",
                        "mf" : "motherfucker",
                        "mfs" : "motherfuckers",
                        "mfw" : "my face when",
                        "mofo" : "motherfucker",
                        "mph" : "miles per hour",
                        "mr" : "mister",
                        "mrw" : "my reaction when",
                        "ms" : "miss",
                        "mte" : "my thoughts exactly",
                        "nagi" : "not a good idea",
                        "nbc" : "national broadcasting company",
                        "nbd" : "not big deal",
                        "nfs" : "not for sale",
                        "ngl" : "not going to lie",
                        "nhs" : "national health service",
                        "nrn" : "no reply necessary",
                        "nsfl" : "not safe for life",
                        "nsfw" : "not safe for work",
                        "nth" : "nice to have",
                        "nvr" : "never",
                        "nyc" : "new york city",
                        "oc" : "original content",
                        "og" : "original",
                        "ohp" : "overhead projector",
                        "oic" : "oh i see",
                        "omdb" : "over my dead body",
                        "omg" : "oh my god",
                        "omw" : "on my way",
                        "p.a" : "per annum",
                        "p.m" : "after midday",
                        "pm" : "prime minister",
                        "poc" : "people of color",
                        "pov" : "point of view",
                        "pp" : "pages",
                        "ppl" : "people",
                        "prw" : "parents are watching",
                        "ps" : "postscript",
                        "pt" : "point",
                        "ptb" : "please text back",
                        "pto" : "please turn over",
                        "qpsa" : "what happens", #"que pasa",
                        "ratchet" : "rude",
                        "rbtl" : "read between the lines",
                        "rlrt" : "real life retweet", 
                        "rofl" : "rolling on the floor laughing",
                        "roflol" : "rolling on the floor laughing out loud",
                        "rotflmao" : "rolling on the floor laughing my ass off",
                        "rt" : "retweet",
                        "ruok" : "are you ok",
                        "sfw" : "safe for work",
                        "sk8" : "skate",
                        "smh" : "shake my head",
                        "sq" : "square",
                        "srsly" : "seriously", 
                        "ssdd" : "same stuff different day",
                        "tbh" : "to be honest",
                        "tbs" : "tablespooful",
                        "tbsp" : "tablespooful",
                        "tfw" : "that feeling when",
                        "thks" : "thank you",
                        "tho" : "though",
                        "thx" : "thank you",
                        "tia" : "thanks in advance",
                        "til" : "today i learned",
                        "tl;dr" : "too long i did not read",
                        "tldr" : "too long i did not read",
                        "tmb" : "tweet me back",
                        "tntl" : "trying not to laugh",
                        "ttyl" : "talk to you later",
                        "u" : "you",
                        "u2" : "you too",
                        "u4e" : "yours for ever",
                        "utc" : "coordinated universal time",
                        "w/" : "with",
                        "w/o" : "without",
                        "w8" : "wait",
                        "wassup" : "what is up",
                        "wb" : "welcome back",
                        "wtf" : "what the fuck",
                        "wtg" : "way to go",
                        "wtpa" : "where the party at",
                        "wuf" : "where are you from",
                        "wuzup" : "what is up",
                        "wywh" : "wish you were here",
                        "yd" : "yard",
                        "ygtr" : "you got that right",
                        "ynk" : "you never know",
                        "zzz" : "sleeping bored and tired"
                        }
            
        sample_typos_slang_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_typos_slang.keys()) + r')(?!\w)')
        sample_acronyms_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_acronyms.keys()) + r')(?!\w)')
        sample_abbr_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_abbr.keys()) + r')(?!\w)')
        
        text = sample_typos_slang_pattern.sub(lambda x: sample_typos_slang[x.group()], text)
        text = sample_acronyms_pattern.sub(lambda x: sample_acronyms[x.group()], text)
        text = sample_abbr_pattern.sub(lambda x: sample_abbr[x.group()], text)
        
        return text

In [35]:
def replace_other(df):
    df['TEXT'] = df['TEXT'].apply(lambda x: other_clean(x))


In [36]:
replace_other(train)
replace_other(test)

In [37]:
train['TEXT'][2154]

' mahouka sure tries hard to make the antidiscrimination course 2 students look like extremists fuck them not being happy with discrimination am i right there is a difference between institutionalized discrimination and personal bigotry they seemed to be fighting against the latter when the former is far more important the former could be changed with progressive policies while the latter cannot obviously bigotry is not ok but like mayumi said you cannot change what an individual feels by force'

In [38]:
train.head()

,ID,TEXT,Depression
0,test_subject1005,kill bill siren was awesome these seriously m...,0
1,test_subject1005,crazy bastard,0
2,test_subject1005,what i hoped for this post something other th...,0
3,test_subject1005,i did not find this scene funny but the fact ...,0
4,test_subject1005,bill from kill bill not saying the five point...,0


## stop_words_removal

In [39]:
def stop_words_removal(df):
    df['TEXT'] = df['TEXT'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    print(df['TEXT'].head())

In [40]:
stop_words_removal(train)
stop_words_removal(test)

0    kill bill siren awesome seriously make week be...
1                                        crazy bastard
2                         hoped post something obvious
3                          find scene funny fact kills
4    bill kill bill saying five point palm explodin...
Name: TEXT, dtype: object
0    today learned saigas species antelope unusuall...
1    bernie sanders say found dead body democratic ...
2    trouble faux iosefka yurie pretty difficult st...
3                      trouble yurie faux iosefka well
4      bloody crow cainhurst sure counts boss fog door
Name: TEXT, dtype: object


## spell_correction

In [41]:
from textblob import TextBlob

In [42]:
def spell_correction(df):
    return df['TEXT'][:5].apply(lambda x: str(TextBlob(x).correct()))

In [43]:
spell_correction(train)
spell_correction(test)
print(train['TEXT'].head())
print(test['TEXT'].head())

0    kill bill siren awesome seriously make week be...
1                                        crazy bastard
2                         hoped post something obvious
3                          find scene funny fact kills
4    bill kill bill saying five point palm explodin...
Name: TEXT, dtype: object
0    today learned saigas species antelope unusuall...
1    bernie sanders say found dead body democratic ...
2    trouble faux iosefka yurie pretty difficult st...
3                      trouble yurie faux iosefka well
4      bloody crow cainhurst sure counts boss fog door
Name: TEXT, dtype: object


# TEXT Preprocessing

# Tokenization

In [47]:
# Tokenizing the tweet base texts.
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
def tokens(df):
    df['tokenized'] = df['TEXT'].apply(word_tokenize)
    print(df['TEXT'].head())
    print(df['tokenized'].head())

In [49]:
tokens(train)
tokens(test)

0    kill bill siren awesome seriously make week be...
1                                        crazy bastard
2                         hoped post something obvious
3                          find scene funny fact kills
4    bill kill bill saying five point palm explodin...
Name: TEXT, dtype: object
0    [kill, bill, siren, awesome, seriously, make, ...
1                                     [crazy, bastard]
2                    [hoped, post, something, obvious]
3                    [find, scene, funny, fact, kills]
4    [bill, kill, bill, saying, five, point, palm, ...
Name: tokenized, dtype: object
0    today learned saigas species antelope unusuall...
1    bernie sanders say found dead body democratic ...
2    trouble faux iosefka yurie pretty difficult st...
3                      trouble yurie faux iosefka well
4      bloody crow cainhurst sure counts boss fog door
Name: TEXT, dtype: object
0    [today, learned, saigas, species, antelope, un...
1    [bernie, sanders, say, found, de

In [50]:
'''def tokens(df):
    return TextBlob(df['TEXT'][1]).words

tokens(train)
tokens(test)'''

"def tokens(df):\n    return TextBlob(df['TEXT'][1]).words\n\ntokens(train)\ntokens(test)"

In [51]:
train.head()

,ID,TEXT,Depression,tokenized
0,test_subject1005,kill bill siren awesome seriously make week be...,0,"[kill, bill, siren, awesome, seriously, make, ..."
1,test_subject1005,crazy bastard,0,"[crazy, bastard]"
2,test_subject1005,hoped post something obvious,0,"[hoped, post, something, obvious]"
3,test_subject1005,find scene funny fact kills,0,"[find, scene, funny, fact, kills]"
4,test_subject1005,bill kill bill saying five point palm explodin...,0,"[bill, kill, bill, saying, five, point, palm, ..."


# Stemming

In [52]:
from nltk.stem import PorterStemmer

def porter_stemmer(text):
    stemmer = nltk.PorterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [53]:
def stemming(df):
    df['porter_stmmer'] = df['tokenized'].apply(lambda x: porter_stemmer(x))

In [54]:
%time

stemming(train)

CPU times: total: 0 ns
Wall time: 0 ns


In [55]:
%time
stemming(test)

CPU times: total: 0 ns
Wall time: 0 ns


In [56]:
train.head()

,ID,TEXT,Depression,tokenized,porter_stmmer
0,test_subject1005,kill bill siren awesome seriously make week be...,0,"[kill, bill, siren, awesome, seriously, make, ...","[kill, bill, siren, awesom, serious, make, wee..."
1,test_subject1005,crazy bastard,0,"[crazy, bastard]","[crazi, bastard]"
2,test_subject1005,hoped post something obvious,0,"[hoped, post, something, obvious]","[hope, post, someth, obviou]"
3,test_subject1005,find scene funny fact kills,0,"[find, scene, funny, fact, kills]","[find, scene, funni, fact, kill]"
4,test_subject1005,bill kill bill saying five point palm explodin...,0,"[bill, kill, bill, saying, five, point, palm, ...","[bill, kill, bill, say, five, point, palm, exp..."


# Part of Speech Tagging (POS Tagging)

In [65]:
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [69]:
from nltk.corpus import wordnet
from nltk.corpus import brown
nltk.download('brown')
wordnet_map = {"N":wordnet.NOUN, 
               "V":wordnet.VERB, 
               "J":wordnet.ADJ, 
               "R":wordnet.ADV
              }
    
train_sents = brown.tagged_sents(categories='news')
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

def pos_tag_wordnet(text, pos_tag_type="pos_tag"):
    pos_tagged_text = t2.tag(text)
    
    # map the pos tagging output with wordnet output 
    pos_tagged_text = [(word, wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    return pos_tagged_text

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [70]:
pos_tag_wordnet(train['tokenized'][2])

[('hoped', 'v'), ('post', 'n'), ('something', 'n'), ('obvious', 'a')]

In [71]:
def pos_tagging(df):
    df['combined_postag_wnet'] = df['tokenized'].apply(lambda x: pos_tag_wordnet(x))

In [72]:
pos_tagging(train)
pos_tagging(test)

In [73]:
train.head()

,ID,TEXT,Depression,tokenized,porter_stmmer,combined_postag_wnet
0,test_subject1005,kill bill siren awesome seriously make week be...,0,"[kill, bill, siren, awesome, seriously, make, ...","[kill, bill, siren, awesom, serious, make, wee...","[(kill, v), (bill, n), (siren, n), (awesome, n..."
1,test_subject1005,crazy bastard,0,"[crazy, bastard]","[crazi, bastard]","[(crazy, a), (bastard, n)]"
2,test_subject1005,hoped post something obvious,0,"[hoped, post, something, obvious]","[hope, post, someth, obviou]","[(hoped, v), (post, n), (something, n), (obvio..."
3,test_subject1005,find scene funny fact kills,0,"[find, scene, funny, fact, kills]","[find, scene, funni, fact, kill]","[(find, v), (scene, n), (funny, n), (fact, n),..."
4,test_subject1005,bill kill bill saying five point palm explodin...,0,"[bill, kill, bill, saying, five, point, palm, ...","[bill, kill, bill, say, five, point, palm, exp...","[(bill, n), (kill, v), (bill, n), (saying, v),..."


# Lemmatization

In [74]:
from textblob import Word
#nltk.download('wordnet')
#nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

In [75]:
# Test without POS Tagging
lemmatizer = WordNetLemmatizer()

def lemmatization(df):
    df['lemmatize_word_wo_pos'] = df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    df['lemmatize_word_wo_pos'] = df['lemmatize_word_wo_pos'].apply(lambda x: [word for word in x if word not in stop])
    df['lemmatize_text_wo'] = [' '.join(map(str, l)) for l in df['lemmatize_word_wo_pos']] # join back to text


In [76]:
lemmatization(train)
lemmatization(test)

In [77]:
print(train["combined_postag_wnet"][0])
print(train["lemmatize_word_wo_pos"][0])

[('kill', 'v'), ('bill', 'n'), ('siren', 'n'), ('awesome', 'n'), ('seriously', 'r'), ('make', 'v'), ('week', 'n'), ('better', 'a'), ('call', 'v'), ('saul', 'n'), ('subreddit', 'n'), ('would', 'n'), ('extremely', 'r'), ('grateful', 'a')]
['kill', 'bill', 'siren', 'awesome', 'seriously', 'make', 'week', 'better', 'call', 'saul', 'subreddit', 'would', 'extremely', 'grateful']


## Lemmatization with pos

In [78]:
# Test with POS Tagging
lemmatizer = WordNetLemmatizer()

def lemmatization_pos(df):
    df['lemmatize_word_w_pos'] = df['combined_postag_wnet'].apply(lambda x: lemmatize_word(x))
    df['lemmatize_word_w_pos'] = df['lemmatize_word_w_pos'].apply(lambda x: [word for word in x if word not in stop]) # double check to remove stop words
    df['lemmatize_text'] = [' '.join(map(str, l)) for l in df['lemmatize_word_w_pos']] # join back to text

In [79]:
lemmatization_pos(train)
lemmatization_pos(test)

In [80]:
print(train["TEXT"][9])
print(train["combined_postag_wnet"][9])
print(train["lemmatize_word_wo_pos"][9])
print(train["lemmatize_word_w_pos"][9])
print(train["lemmatize_text"][9])

seriously cannot stop singing song arizona moon shining
[('seriously', 'r'), ('can', 'n'), ('not', 'n'), ('stop', 'v'), ('singing', 'v'), ('song', 'n'), ('arizona', 'n'), ('moon', 'n'), ('shining', 'n')]
['seriously', 'stop', 'singing', 'song', 'arizona', 'moon', 'shining']
['seriously', 'stop', 'sing', 'song', 'arizona', 'moon', 'shining']
seriously stop sing song arizona moon shining


In [81]:
display(train["TEXT"][0], train["lemmatize_text"][0])
display(train["TEXT"][5], train["lemmatize_text"][5])
display(train["TEXT"][30], train["lemmatize_text"][30])
display(train["TEXT"][15], train["lemmatize_text"][15])
display(train["TEXT"][20], train["lemmatize_text"][20])

'kill bill siren awesome seriously make week better call saul subreddit would extremely grateful'

'kill bill siren awesome seriously make week good call saul subreddit would extremely grateful'

'jump conclusions application'

'jump conclusion application'

'know talking much assure'

'know talk much assure'

'tell us die like dogs'

'tell u die like dog'

'hahaha exactly looking lost boys damn good way'

'hahaha exactly look lose boy damn good way'

In [82]:
train.head()

,ID,TEXT,Depression,tokenized,porter_stmmer,combined_postag_wnet,lemmatize_word_wo_pos,lemmatize_text_wo,lemmatize_word_w_pos,lemmatize_text
0,test_subject1005,kill bill siren awesome seriously make week be...,0,"[kill, bill, siren, awesome, seriously, make, ...","[kill, bill, siren, awesom, serious, make, wee...","[(kill, v), (bill, n), (siren, n), (awesome, n...","[kill, bill, siren, awesome, seriously, make, ...",kill bill siren awesome seriously make week be...,"[kill, bill, siren, awesome, seriously, make, ...",kill bill siren awesome seriously make week go...
1,test_subject1005,crazy bastard,0,"[crazy, bastard]","[crazi, bastard]","[(crazy, a), (bastard, n)]","[crazy, bastard]",crazy bastard,"[crazy, bastard]",crazy bastard
2,test_subject1005,hoped post something obvious,0,"[hoped, post, something, obvious]","[hope, post, someth, obviou]","[(hoped, v), (post, n), (something, n), (obvio...","[hoped, post, something, obvious]",hoped post something obvious,"[hop, post, something, obvious]",hop post something obvious
3,test_subject1005,find scene funny fact kills,0,"[find, scene, funny, fact, kills]","[find, scene, funni, fact, kill]","[(find, v), (scene, n), (funny, n), (fact, n),...","[find, scene, funny, fact, kill]",find scene funny fact kill,"[find, scene, funny, fact, kill]",find scene funny fact kill
4,test_subject1005,bill kill bill saying five point palm explodin...,0,"[bill, kill, bill, saying, five, point, palm, ...","[bill, kill, bill, say, five, point, palm, exp...","[(bill, n), (kill, v), (bill, n), (saying, v),...","[bill, kill, bill, saying, five, point, palm, ...",bill kill bill saying five point palm explodin...,"[bill, kill, bill, say, five, point, palm, exp...",bill kill bill say five point palm exploding h...


In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531394 entries, 0 to 531393
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ID                     531394 non-null  object
 1   TEXT                   531394 non-null  object
 2   Depression             531394 non-null  int64 
 3   tokenized              531394 non-null  object
 4   porter_stmmer          531394 non-null  object
 5   combined_postag_wnet   531394 non-null  object
 6   lemmatize_word_wo_pos  531394 non-null  object
 7   lemmatize_text_wo      531394 non-null  object
 8   lemmatize_word_w_pos   531394 non-null  object
 9   lemmatize_text         531394 non-null  object
dtypes: int64(1), object(9)
memory usage: 40.5+ MB


In [84]:
train[train['TEXT'] == '']

,ID,TEXT,Depression,tokenized,porter_stmmer,combined_postag_wnet,lemmatize_word_wo_pos,lemmatize_text_wo,lemmatize_word_w_pos,lemmatize_text
465,test_subject1085,,0,[],[],[],[],,[],
466,test_subject1085,,0,[],[],[],[],,[],
500,test_subject1103,,0,[],[],[],[],,[],
561,test_subject1103,,0,[],[],[],[],,[],
572,test_subject1103,,0,[],[],[],[],,[],
...,...,...,...,...,...,...,...,...,...,...
530660,train_subject9974,,0,[],[],[],[],,[],
531126,train_subject9974,,0,[],[],[],[],,[],
531246,train_subject9974,,0,[],[],[],[],,[],
531281,train_subject9974,,0,[],[],[],[],,[],


In [85]:
train = train[~((train['TEXT'] == '') |(train['lemmatize_text'] == '') | (train['lemmatize_text_wo'] == '') )]

In [86]:
test = test[~((test['TEXT'] == '') |(test['lemmatize_text'] == '') | (test['lemmatize_text_wo'] == '') )]

In [87]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525700 entries, 0 to 531393
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ID                     525700 non-null  object
 1   TEXT                   525700 non-null  object
 2   Depression             525700 non-null  int64 
 3   tokenized              525700 non-null  object
 4   porter_stmmer          525700 non-null  object
 5   combined_postag_wnet   525700 non-null  object
 6   lemmatize_word_wo_pos  525700 non-null  object
 7   lemmatize_text_wo      525700 non-null  object
 8   lemmatize_word_w_pos   525700 non-null  object
 9   lemmatize_text         525700 non-null  object
dtypes: int64(1), object(9)
memory usage: 44.1+ MB


In [88]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539120 entries, 0 to 544446
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ID                     539120 non-null  object
 1   TEXT                   539120 non-null  object
 2   Depression             539120 non-null  int64 
 3   tokenized              539120 non-null  object
 4   porter_stmmer          539120 non-null  object
 5   combined_postag_wnet   539120 non-null  object
 6   lemmatize_word_wo_pos  539120 non-null  object
 7   lemmatize_text_wo      539120 non-null  object
 8   lemmatize_word_w_pos   539120 non-null  object
 9   lemmatize_text         539120 non-null  object
dtypes: int64(1), object(9)
memory usage: 45.2+ MB


In [89]:
train = train[['ID','TEXT','lemmatize_text','lemmatize_text_wo','Depression']]
test = test[['ID','TEXT','lemmatize_text','lemmatize_text_wo','Depression']]

In [ ]:
'''dft = train.groupby('ID',group_keys=True)['TEXT'].agg(lambda x: ','.join(x.tolist()))
dfd = train.groupby('ID')['Depression'].unique()
dfi = train.groupby('ID')['ID'].unique()'''
df_train = pd.concat ([train.groupby('ID')['ID'].unique() ,  train.groupby('ID',group_keys=True)['lemmatize_text'].agg(lambda x: ','.join(x.tolist())) , train.groupby('ID')['Depression'].unique().astype('int')],axis = 1)


df_test = pd.concat ([test.groupby('ID')['ID'].unique() ,  test.groupby('ID',group_keys=True)['lemmatize_text'].agg(lambda x: ','.join(x.tolist())) , test.groupby('ID')['Depression'].unique().astype('int')],axis = 1)

In [ ]:
df_train.head()

,ID,lemmatize_text,Depression
ID,,,
test_subject1005,[test_subject1005],kill bill siren awesome seriously make week go...,0
test_subject1062,[test_subject1062],wind chill become heat index know wind chill u...,0
test_subject1085,[test_subject1085],exactly know get job bookstore without least k...,0
test_subject1103,[test_subject1103],tell 16 year old would get angry day old infan...,0
test_subject1111,[test_subject1111],"really see greek want rude canadian though,buy...",0


In [ ]:
df_train.describe()

,Depression
count,887.000000
mean,0.152198
std,0.359416
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 887 entries, test_subject1005 to train_subject9974
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              887 non-null    object
 1   lemmatize_text  887 non-null    object
 2   Depression      887 non-null    int32 
dtypes: int32(1), object(2)
memory usage: 24.3+ KB


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 820 entries, subject1010 to subject998
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              820 non-null    object
 1   lemmatize_text  820 non-null    object
 2   Depression      820 non-null    int32 
dtypes: int32(1), object(2)
memory usage: 22.4+ KB


##  export csv

In [193]:
df_train.to_csv('train_clean.csv', index=None)
df_test.to_csv('test_clean.csv', index=None)

df_train_wo.to_csv('train_clean_wo.csv', index=None)
df_test_wo.to_csv('test_clean_wo.csv', index=None)